In [4]:
from minio import Minio
from minio.error import S3Error

client = Minio(
    "localhost:9000",  # Replace with your MinIO server address
    access_key="ml_testing",
    secret_key="ml_testing",
    secure=False  # Set to True if using HTTPS
)

# Create a bucket
bucket_name = "ml-bucket"
try:
    client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
except S3Error as err:
    print(f"Error occurred: {err}")

Bucket 'ml-bucket' created successfully.


In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as pltpoetry 
df = pd.read_pickle('ev_pareto_on_gen.pkl')

In [ ]:
# Split the data into features and target
X = df[["rotorAngle","rotorDiameter","Bias"]]
y = df[['LapTime','Durability','MotorPower','Torque']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the CatBoostRegressor
model = CatBoostRegressor(
    iterations=2000, learning_rate=0.05, depth=4, loss_function='MultiRMSE', verbose=200)

# Fit the model
model.fit(X_train, y_train)

In [ ]:
import plotly.graph_objects as go


column_names = ['LapTime','Durability','MotorPower','Torque']
predictions = model.predict(X_test)
df_predictions = pd.DataFrame(predictions,columns=column_names)

col = 0


fig = go.Figure()
x,y = y_test[column_names[col]],df_predictions[column_names[col]]
fig.add_trace(go.Scatter(x=x, y=y, mode='markers'))
fig.add_trace(go.Scatter(x=[x.min(), x.max()], y=[y.min(), y.max()], mode='lines'))

fig.update_layout(height=250,margin=dict(l=20, r=20, t=20, b=20))
fig.update_layout(
    xaxis_title='Actual',
    yaxis=dict(title='Predicted'),
    template='plotly_dark')
fig.show()